In [38]:
%load_ext autoreload
%autoreload 2

import functools
import gc
import logging
import pickle
import sys
from pathlib import Path
from typing import Dict, List, Tuple, Callable

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#import tensorflow_addons as tfa
from IPython.display import clear_output
#from keras.regularizers import L1L2
from multiprocess import Pool
from multiprocess.dummy import Pool as ThreadPool
from tqdm import tqdm
import lightgbm as lgb


logging.basicConfig(level=logging.DEBUG, format='%(asctime)s :: %(funcName)s :: %(message)s')

from ccf.models_cust import ModelLGB
from ccf.datasets import get_sample, get_initial_setting, get_left_right_name, get_std
from sklearn.metrics import r2_score
from ccf.utils import delete_objs, cuttoff_ts_in_df, natural_reindex, sort_df, create_block_vars_from_separate
from ccf.analytics_cluster import get_analytics_row, get_pairs_rel_analytics, paired_test_greedy_fs
#from ccf.callbacks import FrequencyCallback, FrequencyEpoch
from ccf.preprocess import (
    get_sample_2d_lgb,
    get_sample_2d_bin,
    get_sample_2d_lgb_paired,
    get_sample_2d_no_scale_paired,
)
from tensorflow.keras.callbacks import EarlyStopping
pd.set_option('display.max_columns', 10_000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
base = '../../Storage/alber'
target_type = 'ret_1_10_folds_exp'

list_candidates = pd.read_csv(f'{base}/{target_type}/analytics/one_factor_block_vars_vars_60_corr_05.csv')
list_candidates = list(list_candidates.new_var)


current_feature_list = ['stoch_k_price_21_1']
list_candidates = list(set(list_candidates) - set(current_feature_list))

In [40]:
len(current_feature_list), current_feature_list

(1, ['stoch_k_price_21_1'])

In [43]:
len(list_candidates), list_candidates

(28,
 ['Money',
  'ask_size2',
  'ask_spread',
  'bid_ask_spread2',
  'bid_ask_w_spread2',
  'bid_price2',
  'bid_size2',
  'bid_spread',
  'log_return_mean_price',
  'rel_order_count_1_80',
  'rel_order_count_20_40',
  'rel_order_count_40_80',
  'rel_order_count_5_80',
  'rel_price_10_40',
  'rel_price_1_5',
  'rel_price_40_80',
  'rel_price_5_10',
  'rel_size_10_80',
  'rel_size_40_80',
  'rel_stoch_price_21_1_3',
  'rel_volume_ask',
  'rel_volume_ask_bid',
  'rel_volume_ask_bid1',
  'stoch_d_size_42_3',
  'stoch_k_size_14_1',
  'total_volume',
  'volume_imbalance',
  'wap_balance'])

In [44]:
#current_feature_list = []
folds = []
for i in range(10):
    folds.append([
        Path(f'{base}/{target_type}/X_{i+1}.parquet.gzip'), 
        Path(f'{base}/{target_type}/train_val_{i+1}.parquet.gzip'), 
        Path(f'{base}/{target_type}/val_{i+1}.parquet.gzip'),
        Path(f'{base}/features.parquet.gzip')
    ])


count_obs_train = 1_500_000
count_obs_val_train = 535_000
count_obs_val = None

features_path = Path(f'{base}/features.parquet.gzip')
list_candidates.sort()

selection_rule = {"field_name": "rel_diff_macro_lower_boot_95", "ascending": False}

dict_fields, _, _ = get_initial_setting(
    features_path,
    count_cuttoff = 0
)

experiment_name = f'fs_fs_main_set_1_{target_type}'

analytics_path = Path(f'{base}/{target_type}/analytics/')


get_sample_func = lambda possible_feature_list, base_path, count_obs, scaler, features_path: get_sample_2d_lgb(
    possible_feature_list,
    base_path,
    count_obs,
    features_path,
    categoricals=[],
    experiment_name=None,
    keys=["time"],
)

logging.debug(f"experiment_name == {experiment_name}")
curr_setting = {
    "verbose_eval": 50,
    "num_boost_round": 500,
    "early_stopping_rounds": 50,
    "params": {
        "num_leaves": 131_072,
        "max_bin": 256,
        "learning_rate": 0.01,
        "objective": "regression",
        "metric": "rmse",
        "max_depth": 6,
        "feature_fraction": 1.0,
        "feature_fraction_bynode": 0.6,
        "bagging_fraction": 1.0
    },
}


model_class = lambda train_matrix_shape, name: ModelLGB(
    save_path = Path(f'{base}') / Path("saved_models"), 
    name = name,
    metric = r2_score,
    learning_setting = curr_setting
)

pred_iter_perf=eval(
    pd.read_csv(f'{base}/{target_type}/analytics/block_vars_fs_fs_main_set_0_ret_1_10_folds_exp.csv')\
    .query('best_in_iter == 1')\
    .sort_values(by=['count_vars', 'count_boot_le_one', 'rel_diff_macro_lower_boot_95'], 
                 ascending=[False, True, False]).reset_index(drop=True)\
    .loc[0, "macro"])

2022-10-31 00:48:24,102 :: <module> :: experiment_name == fs_fs_main_set_1_ret_1_10_folds_exp


In [ ]:
paired_test_greedy_fs(
    current_feature_list,
    list_candidates,
    dict_fields,
    folds,
    count_obs_train,
    count_obs_val,
    experiment_name,
    model_class,
    analytics_path,
    selection_rule,
    get_sample_func,
    pred_iter_perf,
    count_obs_val_train,
    print_iteration=True,
    count_iteration=5,
)

2022-11-01 16:31:37,766 :: prepare_output :: 20/21 :: stoch_k_size_14_1 :: 22
2022-11-01 16:31:37,767 :: get_specific_features :: data.columns Index(['stoch_k_price_21_1', 'Money', 'ask_size2', 'ask_spread',
       'bid_ask_spread2', 'bid_ask_w_spread2', 'bid_price2', 'bid_size2',
       'bid_spread', 'log_return_mean_price', 'rel_order_count_1_80',
       'rel_order_count_20_40', 'rel_order_count_40_80',
       'rel_order_count_5_80', 'rel_price_10_40', 'rel_price_1_5',
       'rel_price_40_80', 'rel_price_5_10', 'rel_size_10_80', 'rel_size_40_80',
       'rel_stoch_price_21_1_3', 'rel_volume_ask', 'rel_volume_ask_bid',
       'rel_volume_ask_bid1', 'stoch_d_size_42_3', 'stoch_k_size_14_1',
       'total_volume', 'volume_imbalance', 'wap_balance'],
      dtype='object'), possible_feature_list == ['stoch_k_price_21_1', 'wap_balance', 'volume_imbalance', 'rel_order_count_1_80', 'rel_price_5_10', 'log_return_mean_price', 'bid_spread', 'ask_spread', 'stoch_k_size_14_1']
2022-11-01 16:31:3

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010896 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2148
[LightGBM] [Info] Number of data points in the train set: 1500000, number of used features: 9
[LightGBM] [Info] Start training from score 0.000003
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Lig

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[150]	training's rmse: 0.000737681	valid_1's rmse: 0.000740029
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

2022-11-01 16:35:42,675 :: paired_test_greedy_fs :: After train.
2022-11-01 16:35:42,676 :: get_specific_features :: data.columns Index(['stoch_k_price_21_1', 'Money', 'ask_size2', 'ask_spread',
       'bid_ask_spread2', 'bid_ask_w_spread2', 'bid_price2', 'bid_size2',
       'bid_spread', 'log_return_mean_price', 'rel_order_count_1_80',
       'rel_order_count_20_40', 'rel_order_count_40_80',
       'rel_order_count_5_80', 'rel_price_10_40', 'rel_price_1_5',
       'rel_price_40_80', 'rel_price_5_10', 'rel_size_10_80', 'rel_size_40_80',
       'rel_stoch_price_21_1_3', 'rel_volume_ask', 'rel_volume_ask_bid',
       'rel_volume_ask_bid1', 'stoch_d_size_42_3', 'stoch_k_size_14_1',
       'total_volume', 'volume_imbalance', 'wap_balance'],
      dtype='object'), possible_feature_list == ['stoch_k_price_21_1', 'wap_balance', 'volume_imbalance', 'rel_order_count_1_80', 'rel_price_5_10', 'log_return_mean_price', 'bid_spread', 'ask_spread', 'stoch_k_size_14_1']
2022-11-01 16:35:42,681 :: get_

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[500]	training's rmse: 0.000703638	valid_1's rmse: 0.000707365
Did not meet early stopping. Best iteration is:
[500]	training's rmse: 0.000703638	valid_1's rmse: 0.000707365
(440000,) [ 5.35734696e-04  5.20889647e-04  5.95075369e-04  5.95075369e-04
  5.95075369e-04  5.95075369e-04  5.74409321e-04  5.72819961e-04
  5.72819961e-04  6.15783152e-04  6.15783152e-04  3.37569276e-04
  3.37569276e-04  3.42870480e-04  3.44990956e-04  4.75886889e-04
  4.75886889e-04  5.38420223e-04  6.01483218e-04  6.01483218e-04
  4.79066803e-04  4.79066803e-04  4.79066

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass sample_weight=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass sample_weight=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
2022-11-01 16:35:43,405 :: paired_test_greedy_fs :: performances == [0.034872403282289355, 0.03493560476595747, 0.03482881084691358, 0.03486585517598895, 0.034961446779415706, 0.032021796662922264, 0.032221437354181814, 0.032031297902929134, 0.032053347339843, 0.032176084165249086, 0.015111930226916592, 0.015184198288046957, 0.014914443569628344, 0.015057863044976694, 0.015230519308718216, 0.012078072890632674]
202

delete model 1
delete model 2
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010895 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2145
[LightGBM] [Info] Number of data points in the train set: 1500000, number of used features: 9
[LightGBM] [Info] Start training from score 0.000002
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
